In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


imdb = keras.datasets.imdb(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)


17464789/17464789 [==============================] - 2s 0us/step


In [35]:
max_len = 256
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=0, padding='post', maxlen=max_len)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=0, padding='post', maxlen=max_len)


train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)

In [38]:
class Transformer(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(Transformer, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),])
        self.layer_norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layer_norm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layer_norm2(out1 + ffn_output)

vocab_size = 10000 
embed_dim = 32
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


embed_dim = 32 
num_heads = 2 
ff_dim = 32 

inputs = layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = Transformer(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [39]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])

model.fit(train_data, train_labels, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
625/625 [==============================] - 113s 175ms/step - loss: 0.6554 - accuracy: 0.6093 - val_loss: 0.5350 - val_accuracy: 0.7876
Epoch 2/5
625/625 [==============================] - 108s 173ms/step - loss: 0.3807 - accuracy: 0.8486 - val_loss: 0.3145 - val_accuracy: 0.8680
Epoch 3/5
625/625 [==============================] - 109s 174ms/step - loss: 0.2554 - accuracy: 0.9015 - val_loss: 0.2904 - val_accuracy: 0.8802
Epoch 4/5
625/625 [==============================] - 112s 179ms/step - loss: 0.2073 - accuracy: 0.9205 - val_loss: 0.3008 - val_accuracy: 0.8780
Epoch 5/5
625/625 [==============================] - 109s 174ms/step - loss: 0.1726 - accuracy: 0.9388 - val_loss: 0.3041 - val_accuracy: 0.8828


In [41]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f"Test Accuracy: {test_acc}")
print(f"Test Loss : {test_loss}")

782/782 [==============================] - 53s 67ms/step - loss: 0.3159 - accuracy: 0.8756
Test Accuracy: 0.8755599856376648
Test Loss : 0.31590110063552856
